# Load library

In [1]:
import numpy as np
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from pathlib import Path
import matplotlib
from sklearn.model_selection import train_test_split
from torch.autograd import Variable
import torch.optim as optim
import torchvision

In [2]:
# check version
print("cv2=="+str(cv2.__version__))
print("numpy=="+str(np.__version__))
print("torch=="+str(torch.__version__))
print("matplotlib=="+str(matplotlib.__version__))

cv2==4.8.1
numpy==1.26.1
torch==2.1.0
matplotlib==3.8.1


In [3]:
from utils.FastF import get_magnitude_spectrum
from utils.noise_map import get_noise_map

In [ ]:
bsz = 64
log_interval = 100
gpu = True
device = 'CUDA'

# Data Loader

In [9]:
# test 两个方程可以跑
path = 'data/example.jpeg'
img = cv2.imread(path)
get_magnitude_spectrum(path)
get_noise_map(path)
print("passes")

passes


In [14]:
# test dataset 可以读到数据
root = 'face dataset/test/fake'
images_path = Path(root)
images_list = list(images_path.glob('*.jpg'))
images_list_str = [ str(x) for x in images_list ]
images_list_str
# print(images_list_str[1])
for image_path in images_list_str:
    oroginal = cv2.imread(image_path)
    noise = get_noise_map(image_path)
    magnitude = get_magnitude_spectrum(image_path)
print("passed")

passed


In [15]:
class FaceDataset(Dataset):
    def __init__(self, real_root, fake_root):
        self.samples = list(range(1, 1001))
        self.real_root = real_root
        images_path_real = Path(real_root)
        images_list_real = list(images_path_real.glob('*.jpg'))
        images_list_str_real = [ str(x) for x in images_list_real ]
        true_label = [1 for _ in images_list_real]

        images_path_fake = Path(fake_root)
        images_list_fake = list(images_path_fake.glob('*.jpg'))
        images_list_str_fake = [ str(x) for x in images_list_fake ]
        false_label = [0 for _ in images_list_fake]

        self.images = images_list_str_real+images_list_str_fake
        self.labels = true_label+false_label
        

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        
        oroginal = cv2.imread(image_path)
        noise = get_noise_map(image_path)
        magnitude = get_magnitude_spectrum(image_path)
        label = self.labels[idx]

        sample = {
            "oroginal" : oroginal,
            "noise" : noise,
            "magnitude" : magnitude,
            "label" : label
        }
        return sample


In [ ]:
# table_train,table_test = train_test_split(main_table, test_size=0.2)
train = FaceDataset()
train_loader = DataLoader(train, batch_size=bsz, shuffle=True)
test = FaceDataset()
test_loader = DataLoader(test, batch_size=bsz, shuffle=True)

# Define Model

In [ ]:
model = torchvision.models.vgg19(pretrained=False)
if gpu:
    model = model.to(device)
optimizer = optim.Adagrad(model.parameters())

# Train

In [ ]:
def training(data_loader):
    model.train()
    total_loss = 0

    for iteration, sample in enumerate(data_loader):
        oroginal = Variable(sample['original'])
        noise = Variable(sample['noise'])
        magnitude = Variable(sample['magnitude'])

        label_true = Variable(sample['label'])

    if gpu:
        noise=noise.to(device)
        oroginal=oroginal.to(device)
        magnitude=magnitude.to(device)
        label_true=label_true.to(device)
    
    label = model() # FIXME
    loss = 0 # FIXME
    loss.backward()
    optimizer.step() 
    total_loss += loss.data

    if iteration%log_interval == 0 and iteration > 0:
            # print('total loss: ',str(total_loss))
            cur_loss = total_loss.item() / log_interval
            print('| epoch {:3d} | batches {:5d} | loss {:8.5f}'.format(epoch,iteration,cur_loss))
            total_loss = 0

In [ ]:
def evaluate(data_loader):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for iteration, sample in enumerate(data_loader):
            oroginal = Variable(sample['original'])
            noise = Variable(sample['noise'])
            magnitude = Variable(sample['magnitude'])

            label_true = Variable(sample['label'])

        if gpu:
            noise=noise.to(device)
            oroginal=oroginal.to(device)
            magnitude=magnitude.to(device)
            label_true=label_true.to(device)    

        label = model() # FIXME
        loss = 0        # FIXME
        total_loss += loss.data

    print('RI mse', total_loss.item()/(iteration+1)) # FIXME

In [ ]:
epochs = 2
try:
    for epoch in range(1, epochs+1):
        # epoch_start_time = time.time()
        training(train_loader)
        evaluate(test_loader)
        print('-'*89)
except KeyboardInterrupt:
    print('-'*89)
    print('Existing from training early')